<a href="https://colab.research.google.com/github/ViFLara/Statistics-and-Machine-Learning/blob/master/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

data = pd.read_csv("drive/MyDrive/creditcard.csv")
data.head(10) # amount -> transaction value

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,-1.416907,-0.153826,-0.751063,0.167372,0.050144,-0.443587,0.002821,-0.611987,-0.045575,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,-0.619468,0.291474,1.757964,-1.323865,0.686133,-0.076127,-1.222127,-0.358222,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,-0.705117,-0.110452,-0.286254,0.074355,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,1.017614,0.836390,1.006844,-0.443523,0.150219,0.739453,-0.540980,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [8]:
print(data.isna().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [9]:
n_transactions = data['Class'].count()
n_frauds = data['Class'].sum() # sum of all rows
n_no_frauds = n_transactions - n_frauds
frauds_percentage = n_frauds / n_transactions
no_frauds_percentage = n_no_frauds / n_transactions
print("Number of transactions: ", n_transactions)
print("Number of frauds: ", n_frauds, "%.2f" %(frauds_percentage * 100))
print("Number of transactions without frouds: ", n_no_frauds, "%.2f" %(no_frauds_percentage * 100))

Number of transactions:  284807
Number of frauds:  492 0.17
Number of transactions without frouds:  284315 99.83


In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

def run_validator(x, y):
  
  validator = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for train_id, test_id in validator.split(x, y) :
      x_train, x_test = x[train_id], x[test_id]
      y_train, y_test = y[train_id], y[test_id]
  return x_train, x_test, y_train, y_test

In [11]:
%%time
from sklearn import tree

def run_classifier(classifier, x_train, x_test, y_train):
  
  tree = classifier.fit(x_train, y_train)
  y_pred = tree.predict(x_test) # predict whether transactions are fraud or not
  return y_pred

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 28.6 µs


In [12]:
import matplotlib.pyplot as plt

def save_tree(classifier, name):
  
  plt.figure(figsize=(200,100))
  tree.plot_tree(classifier, filled=True, fontsize=14)
  plt.savefig(name)
  plt.close()

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validate_tree(y_test, y_pred):

  print(accuracy_score(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))
  print(precision_score(y_test, y_pred)) # fraud transactions
  print(recall_score(y_test, y_pred)) # number of fraud hits / number of transactions classified as frauds

In [14]:
# validator execution
x = data.drop('Class', axis=1).values
y = data['Class'].values

x_train, x_test, y_train, y_test = run_validator(x, y)

In [15]:
# classifier execution of decision_tree_classifier
decision_tree_classifier = tree.DecisionTreeClassifier()
y_predict_decision_tree = run_classifier(decision_tree_classifier, x_train, x_test, y_train)

In [16]:
# creation of the decision tree figure
save_tree(decision_tree_classifier, 'tree_decision1.png')

In [17]:
# decision tree validation
validate_tree(y_test, y_predict_decision_tree)

0.9991222218320986
[[28421    11]
 [   14    35]]
0.7608695652173914
0.7142857142857143


In [18]:
print(decision_tree_classifier)
print(decision_tree_classifier.get_depth())

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
21


In [19]:
# classifier execution of decision_tree_classifier
decision_tree_classifier = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_predict_decision_tree = run_classifier(decision_tree_classifier, x_train, x_test, y_train)

In [20]:
# decision tree validation
validate_tree(y_test, y_predict_decision_tree)

0.9994733330992591
[[28430     2]
 [   13    36]]
0.9473684210526315
0.7346938775510204


In [21]:
# classifier execution of decision_tree_classifier
decision_tree_classifier = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_predict_decision_tree = run_classifier(decision_tree_classifier, x_train, x_test, y_train)

In [22]:
# decision tree validation
validate_tree(y_test, y_predict_decision_tree)

0.9993679997191109
[[28426     6]
 [   12    37]]
0.8604651162790697
0.7551020408163265


In [28]:
%%time
# classifier execution of decision_tree_classifier
decision_tree_classifier = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_predict_decision_tree = run_classifier(decision_tree_classifier, x_train, x_test, y_train)

CPU times: user 8.55 s, sys: 9.8 ms, total: 8.56 s
Wall time: 8.53 s


In [24]:
# decision tree validation
validate_tree(y_test, y_predict_decision_tree)

0.999403110845827
[[28429     3]
 [   14    35]]
0.9210526315789473
0.7142857142857143


In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10) # n_estimators -> number of trees
y_predict_random_forest = run_classifier(random_forest_classifier, x_train, x_test, y_train)

CPU times: user 1min 38s, sys: 105 ms, total: 1min 38s
Wall time: 1min 37s


In [32]:
save_tree(random_forest_classifier.estimators_[0], "random_forest1")
save_tree(random_forest_classifier.estimators_[1], "random_forest2")

In [33]:
validate_tree(y_test, y_predict_random_forest)

0.9995435553526912
[[28431     1]
 [   12    37]]
0.9736842105263158
0.7551020408163265


In [35]:
%%time
from sklearn.ensemble import AdaBoostClassifier
adaboost_classifier = AdaBoostClassifier(random_state=0)
y_predict_adaboost = run_classifier(adaboost_classifier, x_train, x_test, y_train)

CPU times: user 1min 43s, sys: 110 ms, total: 1min 43s
Wall time: 1min 43s


In [36]:
save_tree(adaboost_classifier.estimators_[0], "adaboost1")
save_tree(adaboost_classifier.estimators_[1], "adaboost2")

In [37]:
validate_tree(y_test, y_predict_adaboost)

0.9992626663389628
[[28428     4]
 [   17    32]]
0.8888888888888888
0.6530612244897959


In [38]:
%%time
from sklearn.ensemble import AdaBoostClassifier
adaboost_classifier = AdaBoostClassifier(random_state=0, n_estimators=100)
y_predict_adaboost = run_classifier(adaboost_classifier, x_train, x_test, y_train)

CPU times: user 3min 4s, sys: 229 ms, total: 3min 4s
Wall time: 3min 3s


In [39]:
validate_tree(y_test, y_predict_adaboost)

0.999403110845827
[[28426     6]
 [   11    38]]
0.8636363636363636
0.7755102040816326


In [40]:
%%time
from sklearn.ensemble import AdaBoostClassifier
adaboost_classifier = AdaBoostClassifier(random_state=0, n_estimators=200)
y_predict_adaboost = run_classifier(adaboost_classifier, x_train, x_test, y_train)

CPU times: user 7min, sys: 542 ms, total: 7min 1s
Wall time: 6min 59s


In [41]:
validate_tree(y_test, y_predict_adaboost)

0.9995435553526912
[[28429     3]
 [   10    39]]
0.9285714285714286
0.7959183673469388
